In [1]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
from scipy.stats import expon
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
import warnings
warnings.filterwarnings("ignore")

In [16]:
def NIG_characteristic_function(u, T, r, alpha, beta, delta, DELTA):
  """Compute the characteristic function of the NIG distribution.
  Args:
  - u (array): Vector in R^d.
  - T (float): Terminal time.
  - r (float): Short rate.
  - alpha (float): Alpha parameter of the NIG distribution.
  - beta (array): Array of beta values.
  - delta (float): Delta parameter of the NIG distribution.
  - DELTA (array): Covariance matrix.
  Returns:
  - phi (complex): Characteristic function value.
  """
  mu = -delta * (np.sqrt(alpha**2 - np.square(beta)) - np.sqrt(alpha**2 - np.square(beta + 1)))
  phi = np.exp(np.multiply(1j * T, (r + mu) @ u) + delta * T * (np.sqrt(alpha**2 - beta @ DELTA @ beta) - np.sqrt(alpha**2 - (beta + np.multiply(1j, u)) @ DELTA @ (beta + np.multiply(1j, u)))))
  return phi


def fourier_payoff_call_on_min(u):
  """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
  return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_NIG_call_on_min(R):
  """Calculate the integrand for QMC estimation of the rainbow option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
  p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand.real  # Real part of the outcome of Gauss Hermite Quadrature (weighted sum)

def integrand_to_optimize_NIG_basket_put(R):
  """Calculate the integrand for QMC estimation of the rainbow option under VG model.
  Args:
  - R (array): Array of damping parameters.
  Returns:
  - integrand (float): Integrand value.
  """
  d = len(S0)  # Dimensionality
  X0 = np.log(np.divide(S0, d*K))  # Element-wise division
  y = np.multiply(1j, R)
  phi = NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Characteristic function
  p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
  discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  integrand = K * discount * phi * p
  return integrand.real  # Real part of the outcome of Gauss Hermite Quadrature (weighted sum)


def exponential_pdf(x, sigma_IS):
  """Compute the probability density function of the exponential distribution.

  Args:
  - x (float): Value to evaluate the PDF at.
  - sigma_IS (float): Scale parameter of the exponential distribution.

  Returns:
  - pdf (float): PDF value at x.
  """
  return np.exp(-np.abs(x) / sigma_IS) / (2 * sigma_IS)

def oneD_exponential_inverse_cdf(u, sigma_IS):
  """Compute the inverse cumulative distribution function of the exponential distribution (1D).

  Args:
  - u (float): Value to evaluate the CDF at.
  - sigma_IS (float): Scale parameter of the exponential distribution.

  Returns:
  - phi_inv (float): Inverse CDF value.
  """
  phi_inv = np.piecewise(u, [0 < u <= 0.5, 1 > u > 0.5], [lambda u: sigma_IS * np.log(2 * u), lambda u: sigma_IS * np.log(1 / (2 - 2 * u))])
  return phi_inv

def exponential_inverse_cdf(u, sigma_IS):
  """Compute the component-wise inverse cumulative distribution function of the exponential distribution (multidimensional) of. avector
  Args:
  - u (array): vector to evaluate the inverse CDF at
  - sigma_IS (float): Scale parameter of the exponential distribution.
  Returns:
  - inv_cdf (array): Inverse CDF vector
  """
  f = np.vectorize(oneD_exponential_inverse_cdf)
  return f(u, sigma_IS)


def multivariate_laplace_pdf(x, SIGMA_IS, SIGMA_IS_inv):
    """
    Evaluate the probability density function (PDF) of the multivariate Laplace distribution.
    Args:
    - x (array): Values at which to evaluate the PDF.
    - SIGMA_IS (array): Covariance matrix of the Laplace distribution.
    - SIGMA_IS_inv (array): Inverse of the covariance matrix.
    Returns:
    - pdf_value (float): PDF value corresponding to the input x.
    """
    d = len(x)  # Dimension of the random variable
    v = (2 - d) / 2  # Degrees of freedom parameter
    # Evaluate the multivariate Laplace PDF
    PDF = 2 * (2*np.pi)**(-d/2) * (la.det(SIGMA_IS))**(-0.5) * (x @ SIGMA_IS_inv @ x / 2)**(v / 2) * scipy.special.kv(v, np.sqrt(2 * x @ SIGMA_IS_inv @ x))
    return PDF

def fourier_MC_call_on_min_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, SIGMA_IS, alpha_conf, seed):
    """
    Estimate the price of call on min options under the Normal Inverse Gaussian using Monte Carlo simulation with importance sampling in Fourier space.
    Args:
    - S0 (array): Initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - alpha (float): NIG parameter
    - beta (array): Array of beta parameters.
    - delta (float): NIG parameter
    - DELTA (array): Matrix controlling the degree of correlation of assets
    - N (int): Number of Monte Carlo samples.
    - R (array): Vector of damping parameters.
    - SIGMA_IS (array): covariance matrix of the multivariate Laplace distribution
    - alpha_conf (float): Confidence level.
    - seed (int): Seed for the random generator.
    Returns:
    - MC_estimate (float): Estimated price of the call on min option.
    - MC_stat_error (float): Statistical error of the MC estimation.
    """
    np.random.seed(seed)
    # Number of stocks
    dimension = len(S0)
    # Logarithm of the element-wise division
    X0 = np.log(np.divide(S0, K))
    # Modified discount factor
    discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
    # Contains Monte Carlo price estimates
    V_list = np.zeros(N)
    L_IS = la.cholesky(SIGMA_IS)
    SIGMA_IS_inv = la.inv(SIGMA_IS)
    Z = np.random.multivariate_normal(mean = np.zeros(dimension), cov = np.identity(dimension), size = N )  # Independent samples from the multivariate standard normal distribution
    W = np.random.exponential(scale = 1, size = N )  # N samples from the exponential distribution
    multivar_Laplace_samples = np.sqrt(W.reshape(N, 1)) * (L_IS @ (Z.T)).T  # N samples from the multivariate Laplace distribution
    # For each sample
    for n in range(N):
      u = multivar_Laplace_samples[n]  # Sample from the standard normal distribution
      y = u + np.multiply(1j, R)  # Shifting contour of integration by the damping parameters
      phi =  NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      # Evaluate Fourier Transformed Payoff function
      p = fourier_payoff_call_on_min(y)
      # Evaluate the multivariate PDF
      PDF_eval = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      # Compute Monte Carlo estimators
      V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / PDF_eval)

    # Compute the Monte Carlo estimate
    MC_estimate = np.mean(V_list)
    # Compute the statistical error
    C_alpha = norm.ppf(1 - alpha_conf / 2)
    MC_stat_error = C_alpha * np.std(V_list) / np.sqrt(N)
    return MC_estimate, MC_stat_error

def fourier_MC_basket_put_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, SIGMA_IS, alpha_conf, seed):
    """
    Estimate the price of basket put options under the Normal Inverse Gaussian using Monte Carlo simulation with importance sampling in Fourier space.
    Args:
    - S0 (array): Initial stock prices.
    - K (float): Strike price.
    - r (float): Risk-free interest rate.
    - T (float): Time to maturity.
    - alpha (float): NIG parameter
    - beta (array): Array of beta parameters.
    - delta (float): NIG parameter
    - DELTA (array): Matrix controlling the degree of correlation of assets
    - N (int): Number of Monte Carlo samples.
    - R (array): Vector of damping parameters.
    - SIGMA_IS (array): covariance matrix of the multivariate Laplace distribution
    - alpha_conf (float): Confidence level.
    - seed (int): Seed for the random generator.
    Returns:
    - MC_estimate (float): Estimated price of the basket option.
    - MC_stat_error (float): Statistical error of the MC estimation.
    """
    np.random.seed(seed)
    # Number of stocks
    dimension = len(S0)
    # Logarithm of the element-wise division
    X0 = np.log(np.divide(S0, dimension * K))
    # Modified discount factor
    discount = ((2 * np.pi) ** (-dimension)) * np.exp(-r * T) * np.exp(-R @ X0)
    # Contains Monte Carlo price estimates
    V_list = np.zeros(N)
    L_IS = la.cholesky(SIGMA_IS)
    SIGMA_IS_inv = la.inv(SIGMA_IS)
    Z = np.random.multivariate_normal(mean = np.zeros(dimension), cov = np.identity(dimension), size = N )  # Independent samples from the multivariate standard normal distribution
    W = np.random.exponential(scale = 1, size = N )  # N samples from the exponential distribution
    multivar_Laplace_samples = np.sqrt(W.reshape(N, 1)) * (L_IS @ (Z.T)).T  # N samples from the multivariate Laplace distribution
    # For each sample
    for n in range(N):
      u = multivar_Laplace_samples[n]  # Sample from the standard normal distribution
      y = u + np.multiply(1j, R)  # Shifting contour of integration by the damping parameters
      phi =  NIG_characteristic_function(y, T, r, alpha, beta, delta, DELTA)  # Evaluate characteristic function
      # Evaluate Fourier Transformed Payoff function
      p = fourier_payoff_basket_put(y)
      # Evaluate the multivariate PDF
      PDF_eval = multivariate_laplace_pdf(u, SIGMA_IS, SIGMA_IS_inv)
      # Compute Monte Carlo estimators
      V_list[n] = np.real(K * discount * np.exp(1j * u @ X0) * phi * p / PDF_eval)

    # Compute the Monte Carlo estimate
    MC_estimate = np.mean(V_list)
    # Compute the statistical error
    C_alpha = norm.ppf(1 - alpha_conf / 2)
    MC_stat_error = C_alpha * np.std(V_list) / np.sqrt(N)
    return MC_estimate, MC_stat_error

# Pricing call on min options using MC in the Fourier domain

In [20]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
######## NIG Parameters ########
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def rainbow_constraint_1(R):
  return -1*R

def rainbow_constraint_2(R):
  return -1 - np.sum(R)

cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_call_on_min, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-8.07410476 -8.07410476 -8.07438136]


In [21]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
######## NIG Parameters ########
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### MC parameters ###############
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
SIGMA_IS  = np.sqrt(2 / (delta**2 * T**2)) * la.inv(DELTA)
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_call_on_min_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 0.49227 
MC relative statistical error =  0.0513


# Pricing basket put options using MC in the Fourier domain

In [22]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
######## NIG Parameters ########
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def NIG_constraint(R):
  return alpha**2 - (beta- R) @ DELTA @ (beta - R)

def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': NIG_constraint},
        {'type': 'ineq', 'fun': basket_put_constraint},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_NIG_basket_put, constraints = cons, x0 = R_init , method = "Trust-Constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [3.52609742 3.52609733 3.52609679]


In [23]:
############### Model and payoff parameters ###############
######## Payoff Parameters ########
dimension = 3
S0 = 100 * np.ones(dimension)
K = 100
r = 0
T = 1
######## NIG Parameters ########
alpha= 12
beta = -3 * np.ones(dimension)
delta= 0.2
DELTA = np.identity(dimension)

############### MC parameters ###############
N = 10**5 # number of MC sample paths
alpha_conf = 0.05 # confidence level for MC statistical error estimation
seed = 100 # random seed for reproducibility of results.
SIGMA_IS  = np.sqrt(2 / (delta**2 * T**2)) * la.inv(DELTA)
MC_Fourier_price_estimate, MC_Fourier_stat_error = fourier_MC_basket_put_NIG_pricer(S0, K, r, T, alpha, beta, delta, DELTA, N, R, SIGMA_IS, alpha_conf, seed)
print("MC price estimate =", round(MC_Fourier_price_estimate, 5),"\nMC relative statistical error = ", round(MC_Fourier_stat_error / MC_Fourier_price_estimate,4) )

MC price estimate = 3.32779 
MC relative statistical error =  0.0092
